## TensorFlow Serving Deployment Test
Notebook ini digunakan untuk menguji apakah model TensorFlow Serving 

yang sudah dideploy di Railway bisa menerima request dan mengembalikan hasil prediksi.

In [40]:
# Import library
import tensorflow as tf
import tensorflow_transform as tft
import pandas as pd
import numpy as np
import requests
import base64

In [41]:
# Endpoint TF Serving
TF_SERVING_URL = "https://income-prediction-model-production-3bbd.up.railway.app/v1/models/income-prediction-model:predict"

# Path transform graph hasil pipeline
TRANSFORM_GRAPH_DIR = "output/jasmeinalbaar-pipeline/Transform/transform_graph/5"

# Load transform output
tf_transform_output = tft.TFTransformOutput(TRANSFORM_GRAPH_DIR)

# Contoh raw data
raw_data = pd.DataFrame([
    [90, "?", 77053, "HS-grad", 9, "Widowed", "?", "Not-in-family", "White", "Female", 0, 4356, 40, "United-States"],
    [82, "Private", 132870, "HS-grad", 9, "Widowed", "Exec-managerial", "Not-in-family", "White", "Female", 0, 4356, 18, "United-States"]
], columns=[
    "age","workclass","fnlwgt","education","education.num","marital.status","occupation","relationship",
    "race","sex","capital.gain","capital.loss","hours.per.week","native.country"
])

In [42]:
# Transform raw data menjadi tf.Example menggunakan transform graph
def transform_to_example(tf_transform_output, df):
    transformed_examples = []

    int_features = ["age","fnlwgt","education.num","capital.gain","capital.loss","hours.per.week"]
    cat_features = [c for c in df.columns if c not in int_features]

    input_dict = {}
    for col in df.columns:
        if col in int_features:
            input_dict[col] = df[col].astype(np.int64).values.reshape(-1,1)
        else:
            input_dict[col] = df[col].astype(str).values.reshape(-1,1)

    # Transform
    transformed_features = tf_transform_output.transform_raw_features(input_dict)

    # Convert ke numpy
    transformed_features_np = {k: v.numpy() if isinstance(v, tf.Tensor) else np.array(v)
                               for k, v in transformed_features.items()}

    # Loop per baris
    for i in range(len(df)):
        features_proto = {}
        for key, value in transformed_features_np.items():
            # scalar
            if value.ndim == 1 or value.shape[1] == 1:
                features_proto[key] = tf.train.Feature(float_list=tf.train.FloatList(value=[float(value[i])]))
            else:
                features_proto[key] = tf.train.Feature(float_list=tf.train.FloatList(value=value[i].tolist()))
        example = tf.train.Example(features=tf.train.Features(feature=features_proto))
        transformed_examples.append(base64.b64encode(example.SerializeToString()).decode("utf-8"))

    return transformed_examples

In [43]:
# Transform dan serialisasi ke base64
serialized_base64_examples = transform_to_example(tf_transform_output, raw_data)

# Prepare payload untuk TF Serving
payload = {
    "instances": [{"b64": s} for s in serialized_base64_examples]
}

# Kirim request ke TF Serving
response = requests.post(TF_SERVING_URL, json=payload)
result = response.json()

# Cek hasil
print(result)

INFO:tensorflow:struct2tensor is not available.
INFO:tensorflow:tensorflow_decision_forests is not available.
INFO:tensorflow:tensorflow_text is not available.
{'predictions': [[0.14693509], [0.209623426]]}


In [44]:
# Ambil probabilitas kelas >50K
pred_probs = np.array([item[0] for item in result["predictions"]])

# Hitung probabilitas kelas <=50K
pred_probs_le50 = 1 - pred_probs

# Tentukan label prediksi
pred_labels = np.where(pred_probs > 0.5, ">50K", "<=50K")

# Tampilkan hasil
for i, (p_le50, p_gt50, label) in enumerate(zip(pred_probs_le50, pred_probs, pred_labels), 1):
    # Tentukan simbol untuk probabilitas lebih tinggi
    sym_le50 = "✅" if p_le50 > p_gt50 else ""
    sym_gt50 = "✅" if p_gt50 > p_le50 else ""

    # Simbol untuk prediksi final
    sym_label = "👈"

    print(f"Data ke-{i}:")
    print(f"  Probabilitas <=50K: {p_le50:.3f} {sym_le50}")
    print(f"  Probabilitas >50K : {p_gt50:.3f} {sym_gt50}")
    print(f"  Prediksi final      : {label} {sym_label}\n")

Data ke-1:
  Probabilitas <=50K: 0.853 ✅
  Probabilitas >50K : 0.147 
  Prediksi final      : <=50K 👈

Data ke-2:
  Probabilitas <=50K: 0.790 ✅
  Probabilitas >50K : 0.210 
  Prediksi final      : <=50K 👈

